In [ ]:
# 1. For reading:

# a WFDBrecord object with signals == None.  

item1 = rdheader('headername') # Function belongs to the module

# a WFDBrecord object with signals != None unless it is a 0 channel file. 
item1 = rdsamp('recordname') # Function belongs to the module. 

# Module function
def rdsamp(recordname):
    item1 = rdheader(recordname)
    item1.signals = rddats(item1.filenames)
    return item1


# Module function
def rdheader(recordname):
    f=open()
    return WFDBrecord

# Module function
def rddats(filenames):
    signals = np.fromfile(r)
    return signals



In [ ]:

    
# Write all the dat files for a record. 
# Method belongs to the WFDBrecord class. NOT a function of the module. Does check the signal.  
def wrdats(self): # Don't need record names. The object fields specify. 
    
    # Perform extra check of the signal itself in terms of its values for writing. 
    # Get the final digital values to write to the file. 
    digitalwritesigs = self.checkwritesignals()
    
    # Get a list of the dat files to be written and their formats
    datfiles, datchannels = orderedsetlist(self.filename)
    formats = orderedsetlist(self.fmt)[0]
    
    for i in range(0, len(datfiles)):
        # To do: Consider multi samples per frame writing. 
        wrdatfile(datfiles[i], formats[i], self.signals[:,datchannels[i][0]:(datchannels[i][-1]+1)])
    

    




    
        

  
    


    
# Retrieve a copy of the wfdb signals field, either digital or physical. 
# Returned array will NOT be a reference to the same object.
# Method of WFDBrecord
def getsignalscopy(self, physical = 1)
    
    if not self.signals:
        sys.exit('WFDBrecord has no signals')
    
    # Return the physical signals
    if physical:
        if self.p_signals != None:
            return self.psignals.copy()
        # Have to do dac
        elif self.d_dignals != None:
            return dac(self.d_signals)
        else:
            sys.exit('The WFDBrecord object contains neither p_signals nor d_signals')
    # Return the digital signals
    else:
        if self.d_signals != None:
            return self.dsignals.copy()
        # Have to do adc
        elif self.a_signals != None:
            return adc(self.a_signals)
        else:
            sys.exit('The WFDBrecord object contains neither p_signals nor d_signals')


# Merge the ordered dictionaries in a list into one ordered dictionary. 
def _mergeODlist(ODlist):
    mergedOD=ODlist[0].copy()
    for od in ODlist[1:]:
        mergedOD.update(od)
    return mergedOD
                  

            

In [ ]:
def orderedsetlist(fulllist):
    uniquelist = []
    original_inds = {}

    for i in range(0, len(fulllist)):
        item = fulllist[i]
        # new item
        if item not in uniquelist:
            uniquelist.append(item)
            original_inds[item] = [i]
        # previously seen item
        else:
            original_inds[item].append(i)
    return uniquelist, original_inds





In [ ]:
import numpy as np
fname = ['100.dat','100.dat','100.xyz','100.abc']
fmt = ['212','212','16','16']
sig = np.zeros([50,4])
sig[:,0] = 9
sig[:,1] = 10
sig[:,2:4] = 11

filenames, datchannels = orderedsetlist(fname)


In [ ]:
print(filenames)
print(datchannels)
print(fmt)
print(sig)



In [ ]:
for i in range(0, len(filenames)):
    print(filenames[i]) 
    print(fmt[min(datchannels[filenames[i]])])
    print(sig[:, min(datchannels[filenames[i]]):max(datchannels[filenames[i]])+1])

In [ ]:
datchannels[]

In [ ]:

# Write a dat file.
def wrdatfile(filename, fmt, d_signals):
    f=open(filename,'wb')
    
    # All bytes are written one at a time
    # to avoid endianness issues.

    nsig = d_signals.shape[1]

    if fmt == '80':
        # convert to 8 bit offset binary form
        d_signals = d_signals + 128
        # Convert to unsigned 8 bit dtype to write
        bwrite = d_signals.astype('uint8')
    
    elif fmt == '16':
        # convert to 16 bit two's complement 
        d_signals[d_signals<0] = d_signals[d_signals<0] + 65536
        # Split samples into separate bytes using binary masks
        b1 = signals & [255]*nsig
        b2 = ( signals & [65280]*nsig ) >> 8
        # Interweave the bytes so that the same samples' bytes are consecutive 
        b1 = b1.reshape((-1, 1))
        b2 = b2.reshape((-1, 1))
        bwrite = np.concatenate((b1, b2), axis=1)
        bwrite = bwrite.reshape((1,-1))[0]
        # Convert to unsigned 8 bit dtype to write
        bwrite = bwrite.astype('uint8')

    elif fmt == '24':
        # convert to 24 bit two's complement 
        d_signals[d_signals<0] = d_signals[d_signals<0] + 16777216
        # Split samples into separate bytes using binary masks
        b1 = signals & [255]*nsig
        b2 = ( signals & [65280]*nsig ) >> 8
        b3 = ( signals & [16711680]*nsig ) >> 16
        # Interweave the bytes so that the same samples' bytes are consecutive 
        b1 = b1.reshape((-1, 1))
        b2 = b2.reshape((-1, 1))
        b3 = b3.reshape((-1, 1))
        bwrite = np.concatenate((b1, b2, b3), axis=1)
        bwrite = bwrite.reshape((1,-1))[0]
        # Convert to unsigned 8 bit dtype to write
        bwrite = bwrite.astype('uint8')
    
    elif fmt == '32':
        # convert to 32 bit two's complement 
        d_signals[d_signals<0] = d_signals[d_signals<0] + 4294967296
        # Split samples into separate bytes using binary masks
        b1 = signals & [255]*nsig
        b2 = ( signals & [65280]*nsig ) >> 8
        b3 = ( signals & [16711680]*nsig ) >> 16
        b4 = ( signals & [4278190080]*nsig ) >> 24
        # Interweave the bytes so that the same samples' bytes are consecutive 
        b1 = b1.reshape((-1, 1))
        b2 = b2.reshape((-1, 1))
        b3 = b3.reshape((-1, 1))
        b4 = b4.reshape((-1, 1))
        bwrite = np.concatenate((b1, b2, b3, b4), axis=1)
        bwrite = bwrite.reshape((1,-1))[0]
        # Convert to unsigned 8 bit dtype to write
        bwrite = bwrite.astype('uint8')
    else:
        sys.exit('This library currently only supports the following formats: 80, 16, 24, 32')
    # Write the file
    bwrite.tofile(f)

    f.close()

In [ ]:
# fmt 8
signals8 = np.array([[1,20],[66, 127], [-1,-20], [-127, -128]])
wrdatfile('test80.dat', '80', signals8)
lines = ['test80 2 250 3', 'test80.dat 80 100(0)/mv', 'test80.dat 80 100(0)/mv', 'test80.dat 80 100(0)/mv', 'test80.dat 80 100(0)/mv']
linestofile('test80.hea', lines)

signals = np.array([[1,20],[10000, 20000], [-1,-20], [-10000, -20000]])
# fmt 16
wrdatfile('test16.dat', '16', signals)
lines = ['test16 2 250 3', 'test16.dat 16 100(0)/mv', 'test16.dat 16 100(0)/mv', 'test16.dat 16 100(0)/mv', 'test16.dat 16 100(0)/mv']
linestofile('test16.hea', lines)


# fmt 24
wrdatfile('test24.dat', '24', signals)
lines = ['test24 2 250 3', 'test24.dat 24 100(0)/mv', 'test24.dat 24 100(0)/mv', 'test24.dat 24 100(0)/mv', 'test24.dat 24 100(0)/mv']
linestofile('test24.hea', lines)

# fmt 32
wrdatfile('test32.dat', '32', signals)
lines = ['test32 2 250 3', 'test32.dat 32 100(0)/mv', 'test32.dat 32 100(0)/mv', 'test32.dat 32 100(0)/mv', 'test32.dat 32 100(0)/mv']
linestofile('test32.hea', lines)

In [ ]:
print(signals)